## Rate version 

In [ ]:
def dIBDRdt_Rate(t, I0, B0, D0, R0, params_dict): # calculating time derivative at time step t 
    inp = np.zeros(len(params_dict['IBDR_E_duration'])) # defining vector of external input at this timestep
    
    for ww in range(0, len(inp)): # calculating external input at this timestep. Input may be simple (from time step 0 to T_off) or complex (with delay from time step 0 to T_on)
        if t <= params_dict['IBDR_E_duration'][ww]:
            inp[ww] = params_dict['IBDR_E_amplitude'][ww]
        else:
            if params_dict['Complex_input']=='yes':
                if t <= params_dict['IBDR_E_duration_Complex'][ww]: 
                    inp[ww] = params_dict['IBDR_E_amplitude_Complex'][ww]
                    
    S = SeizureBurden(I0, R0, params_dict)
    dIdt = 1 / params_dict['tau_I'] * (-I0 + params_dict['k_BI']*B0 + inp[0])
    dBdt = 1 / params_dict['tau_B'] * (-B0 + params_dict['k_IB'] *I0 + S + inp[1])
    dDdt = 1 / params_dict['tau_D'] * (
                (1 - D0 / params_dict['D_m']) * params_dict['k_ID'] * np.max([0, I0 - params_dict['Theta']]) +
                inp[2])
    dRdt = 1 / params_dict['tau_R'] * (-R0 + params_dict['k_BR']*B0 + params_dict['k_DR']*D0 + inp[3])
    return ([dIdt, dBdt, dDdt, dRdt])

In [ ]:
def SeizureBurden(I,R, par_dict): # defining seizure burden function (Eq.4) depending on neuroinflammation and circuit remodeling
    return par_dict['K_SB']*(np.exp(par_dict['k_IS']*I*I+par_dict['k_RS']*R)-1)\
                           /(np.exp(par_dict['k_IS']*I*I+par_dict['k_RS']*R)+1)

In [ ]:
def fun_dbdt(x): # function db/dt, where D=D_const; R(t) is expressed as R=kappa_BR*B+kappa_DR*D_const; I is expressed as I=kappa_BI*B
...     return -x+k_IB*k_BI*x+K_SB*(np.exp(k_IS*k_BI*k_BI*x*x+k_RS*k_BR*x+k_RS*k_DR*D_const)-1)\
                                     /(np.exp(k_IS*k_BI*k_BI*x*x+k_RS*k_BR*x+k_RS*k_DR*D_const)+1)

## Stochastic version

In [ ]:
def dIBDRdt_Stoch(t, I0, B0, D0, R0, S0, lambda0, t_lastseiz , params):
        dt=params['dt']
        T_seiz=params['T_seiz']
        
        if S0 == 0:
            if np.random.rand(1) > lambda0 * dt:
                S = 0
                dIBDRdt = dIBDRdt_NoSeiz(t, I0, B0, D0, R0, params)
            else:
                S = 2
                t_lastseiz = t
                dIBDRdt = dIBDRdt_Seiz(t, I0, B0, D0, R0, params)
        else:
            if t > t_lastseiz + T_seiz - dt:
                if np.random.rand(1) > lambda0 * dt:
                    S = 0
                    dIBDRdt = dIBDRdt_NoSeiz(t, I0, B0, D0, R0, params)
                else:
                    S = 2
                    t_lastseiz = t
                    dIBDRdt = dIBDRdt_Seiz(t, I0, B0, D0, R0, params)
            else:
                S = 1
                dIBDRdt = dIBDRdt_Seiz(t, I0, B0, D0, R0, params)
                
        return(dIBDRdt[0], dIBDRdt[1], dIBDRdt[2], dIBDRdt[3], S, t_lastseiz)

In [ ]:
def dIBDRdt_NoSeiz(t, I0, B0, D0, R0, params_dict): # calculating time derivative at time step t for case no seizure occuring on this time step
    inp = np.zeros(len(params_dict['IBDR_E_duration'])) # defining vector of external input at this timestep
    for ww in range(0, len(inp)): # calculating external input at this timestep. Input may be simple (from time step 0 to T_off) or complex (with delay from time step 0 to T_on)
        if t <= params_dict['IBDR_E_duration'][ww]:
            inp[ww] = params_dict['IBDR_E_amplitude'][ww]
        else:
            if params_dict['Complex_input']=='yes':
                if t <= params_dict['IBDR_E_duration_Complex'][ww]: 
                    inp[ww] = params_dict['IBDR_E_amplitude_Complex'][ww]
            
    dIdt = 1 / params_dict['tau_I'] * (-I0 + params_dict['k_BI']*B0 + inp[0])
    dBdt = 1 / params_dict['tau_B'] * (-B0 + params_dict['k_IB'] *I0 + inp[1])
    dDdt = 1 / params_dict['tau_D'] * (
                (1 - D0 / params_dict['D_m']) * params_dict['k_ID'] * np.max([0, I0 - params_dict['Theta']]) +
                inp[2])
    dRdt = 1 / params_dict['tau_R'] * (-R0 + params_dict['k_BR']*B0 + params_dict['k_DR']*D0 + inp[3])
    return ([dIdt, dBdt, dDdt, dRdt])

In [ ]:
def dIBDRdt_Seiz(t, I0, B0, D0, R0, params_dict): # calculating time derivative at time step t for case of seizure occuring on this time step
    inp = np.zeros(len(params_dict['IBDR_E_duration'])) # defining vector of external input at this timestep
    for ww in range(0, len(inp)): # calculating external input at this timestep. Input may be simple (from time step 0 to T_off) or complex (with delay from time step 0 to T_on)
        if t <= params_dict['IBDR_E_duration'][ww]:
            inp[ww] = params_dict['IBDR_E_amplitude'][ww]
        else:
            if params_dict['Complex_input']=='yes':
                if t <= params_dict['IBDR_E_duration_Complex'][ww]: 
                    inp[ww] = params_dict['IBDR_E_amplitude_Complex'][ww]
            
    dIdt = 1 / params_dict['tau_I'] * (-I0 + params_dict['k_BI']*B0 + inp[0])
    dBdt = 1 / params_dict['tau_B'] * (-B0 + params_dict['k_IB'] *I0 + params_dict['k_SB']+ inp[1])
    dDdt = 1 / params_dict['tau_D'] * (
                (1 - D0 / params_dict['D_m']) * params_dict['k_ID'] * np.max([0, I0 - params_dict['Theta']]) +
                inp[2])
    dRdt = 1 / params_dict['tau_R'] * (-R0 + params_dict['k_BR']*B0 + params_dict['k_DR']*D0 + inp[3])
    return ([dIdt, dBdt, dDdt, dRdt])

In [ ]:
def SeizureRate(I,R, par_dict): # defining seizure burden function (Eq. 2) depending on neuroinflammation and circuit remodeling
    return par_dict['lambda_max']*(np.exp(par_dict['k_IS']*I*I+par_dict['k_RS']*R)-1)\
                                 /(np.exp(par_dict['k_IS']*I*I+par_dict['k_RS']*R)+1)